In [32]:
!pip install -q evaluate 
!pip install -q rouge-score

In [30]:
import pandas as pd
import re 
import evaluate

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from evaluate_model import generate_predictions

In [3]:
# Load the model
tokenizer = AutoTokenizer.from_pretrained("jeffreykthomas/llama-mental-health")
model = AutoModelForCausalLM.from_pretrained("jeffreykthomas/llama-mental-health")

In [26]:
# Testing prompt
user_token = '<user>'
agent_token = '<agent>'
prompt = user_token+' Ive been feeling down for weeks now, but I guess its just my dysthymia acting up again. Ill try to keep a positive attitude, but its hard some days. '+agent_token
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_new_tokens=100)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [27]:
output

"<user> Ive been feeling down for weeks now, but I guess its just my dysthymia acting up again. Ill try to keep a positive attitude, but its hard some days. <agent> I'm sorry to hear that you're feeling down. It's normal to feel down. It's normal to feel down because we all feel down at times. It's normal to feel down because we all have a part to play in our lives. It's normal to feel down because we all have a part to play in our communities. It's normal to feel down because we all have a part to play in our communities. It's normal"

In [115]:
#load all csv into a single dataframe
import glob

files = glob.glob('data/datasets/RogueEval/*.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index = True)

In [129]:
# Removes the system instruction from the text
def remove_sys_instr(text):
    text = text.strip()
    text = text.replace("\n", "")
    clean = re.sub(r'<<SYS>>(.*?)<</SYS>>', '', text)
    return clean

# Extracts the instruction as input
def extract_input(text):
    start = '[INST]'
    end = '[/INST]'
    idx1 = text.index(start)
    idx2 = text.index(end)
    input = text[idx1 + len(start) + 1: idx2]
    return input.strip()

# Extracts the output as reference
def extract_reference(text):
    reference = text.replace(extract_input(text), "")
    reference = reference.replace('[INST]', "")
    reference = reference.replace('[/INST]', "")
    return reference.strip()

def calculate_predictions(text):
    user_token = '<user> '
    agent_token = ' <agent>'
    text = user_token + text + agent_token
    inputs = tokenizer(text, return_tensors = "pt")
    # Generate
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=100)
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return output

def clean_predictions(text):
    #Remove User Input
    text = re.sub(r'<user>(.*?)<agent>', "", text)
    return text

In [116]:
# Seperate Inputs and References as texts
df['text'] = df['text'].apply(lambda x: remove_sys_instr(x))
df['Input'] = df['text'].apply(lambda x: extract_input(x))
df['Reference'] = df['text'].apply(lambda x: extract_reference(x))
df.head()

,text,Input,Reference
0,[INST] I can't decide what to wear today. I n...,I can't decide what to wear today. I need you ...,"As an expert in mental health, I would like to..."
1,"[INST] Please help me with this task, I canno...","Please help me with this task, I cannot do it ...",Thank you for reaching out for support. It tak...
2,"[INST] Hey there, I'm really struggling with ...","Hey there, I'm really struggling with my addic...","Hello there! As an expert in mental health, I'..."
3,"[INST] Hi, I'm struggling with my addiction a...","Hi, I'm struggling with my addiction and feel ...",the following is a response to the user messag...
4,"[INST] Hi, I'm really struggling with my addi...","Hi, I'm really struggling with my addiction ri...",Sorry to hear that you're struggling with addi...


In [123]:
# Generate Predictions based on Inputs
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    df['Predictions'] = df['Input'].apply(lambda x: calculate_predictions(x))
    
df.head()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

,text,Input,Reference,Predictions
0,[INST] I can't decide what to wear today. I n...,I can't decide what to wear today. I need you ...,"As an expert in mental health, I would like to...",<user> I can't decide what to wear today. I ne...
1,"[INST] Please help me with this task, I canno...","Please help me with this task, I cannot do it ...",Thank you for reaching out for support. It tak...,"<user> Please help me with this task, I cannot..."
2,"[INST] Hey there, I'm really struggling with ...","Hey there, I'm really struggling with my addic...","Hello there! As an expert in mental health, I'...","<user> Hey there, I'm really struggling with m..."
3,"[INST] Hi, I'm struggling with my addiction a...","Hi, I'm struggling with my addiction and feel ...",the following is a response to the user messag...,"<user> Hi, I'm struggling with my addiction an..."
4,"[INST] Hi, I'm really struggling with my addi...","Hi, I'm really struggling with my addiction ri...",Sorry to hear that you're struggling with addi...,"<user> Hi, I'm really struggling with my addic..."


In [131]:
df['Predictions'] = df['Predictions'].apply(lambda x: clean_predictions(x))

In [139]:
df.Predictions[0]

" I'm sorry to hear that you are struggling with this. I'm glad you are reaching out for help. I'm glad you are willing to try on clothes that you may not have been able to find on your own. I think it would be helpful to have a conversation with you about what you are looking for and what you are willing to do to find it. I think it would be helpful to have a conversation with you about what you are looking for and what you are willing"

In [132]:
df.to_csv('predictions.csv')

In [137]:

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df['Predictions'], references=df['Reference'])
print(results)


{'rouge1': 0.20121440784400918, 'rouge2': 0.04738159572334236, 'rougeL': 0.1369741978156901, 'rougeLsum': 0.13703105940505034}


In [26]:
df_comparison = pd.read_csv('data/datasets/RogueEval/predictions_with_comparison.csv')

In [27]:
df_comparison.head()

,Unnamed: 0,text,Input,Reference,Predictions,Comparison_Predictions
0,1505,[INST] I can't believe I didn't arrange these...,I can't believe I didn't arrange these books i...,1. It's understandable that disorganized books...,"Hi, I'm sorry you're having this problem. I'm...",It sounds like you have some issues with your ...
1,764,[INST] I can't stop thinking about what happe...,I can't stop thinking about what happened. It'...,response: Sorry to hear that you're going ...,I'm sorry to hear that you're struggling with...,It sounds like you're experiencing a lot of di...
2,2938,"[INST] I'm sorry, I can't talk about it right...","I'm sorry, I can't talk about it right now. Th...",efname: Mental Health Expert message: I...,I'm sorry to hear that you're struggling with...,I understand. Is there anything else you'd lik...
3,2872,[INST] I've been feeling really down lately a...,I've been feeling really down lately and I can...,I'm really sorry to hear that you're feeling t...,I'm sorry to hear that you're feeling this wa...,It sounds like you're going through a tough ti...
4,35,[INST] I can't stop thinking about the accide...,I can't stop thinking about the accident. I se...,Sorry to hear that you're going through this. ...,I'm sorry to hear about your anxiety. It soun...,It sounds like you're experiencing anxiety and...


In [28]:
df_comparison['Comparison_Predictions'].iloc[0]

"It sounds like you have some issues with your book order, which could be causing stress and frustration. Here are some steps you can take to improve your order:\n\n1. Check your library catalog: Make sure your book card number is on your library card, as this will help you locate the correct books.\n\n2. Review the cart history: Go through your library card history to see if there are any books you haven't added yet. If you don't see them, check with your librarian or a staff member to double-check.\n\n3. Get rid of books you no longer want: You may find some books you no longer need that you don't want to keep. Ask your librarian for advice on how to delete them.\n\n4. Sort your items by genre or subject: Depending on what kind of books you have, you might be able to sort them into different sections, such as fiction, non-fiction, or children's books. This can help you quickly locate the books you need.\n\n5. Update your library card information: Double-check your library card inform

In [34]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=df_comparison['Comparison_Predictions'], references=df_comparison['Reference'])

In [35]:
results

{'rouge1': 0.30572842710314563,
 'rouge2': 0.08522015655014051,
 'rougeL': 0.1487199951268262,
 'rougeLsum': 0.18706441463955678}